In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as T
import random

from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.optim import Adam, SGD, lr_scheduler
from torch.autograd import Variable

from random import randint, shuffle

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def imageshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
d_mean = [0.485, 0.456, 0.406]
t_std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Resize(100),
    transforms.CenterCrop(100),
    transforms.ToTensor(),
    #transforms.Normalize(d_mean, t_std),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [ ]:
N_CLASSES = 9
class mush_rec_nn_old(nn.Module):
    def __init__(self):
        super(mush_rec_nn_old, self).__init__()
        
        k_s = 3
        pad = 0

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=k_s, stride=1, padding=pad)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(12)
        
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=k_s, stride=1, padding=pad)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(24)
        
        self.cnn3 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=k_s, stride=1, padding=pad)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(48)
        
        self.cnn4 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=k_s, stride=1, padding=pad)
        self.pool4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(96)
        
        self.cnn5 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=k_s, stride=1, padding=pad)
        self.pool5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(192)
        
        linr = 192 * 1 * 1
        linr2 = linr
        self.fc1 = nn.Linear(linr, linr)
        self.fc2 = nn.Linear(linr, 192)
        self.dp1 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(192, N_CLASSES)
    
    def forward(self, input):
        output = self.pool1(F.relu(self.bn1(self.cnn1(input))))
        output = self.pool2(F.relu(self.bn2(self.cnn2(output))))  
        output = self.pool3(F.relu(self.bn3(self.cnn3(output))))
        output = self.pool4(F.relu(self.bn4(self.cnn4(output))))  
        output = self.pool5(F.relu(self.bn5(self.cnn5(output))))
        
        output = output.view(-1, 192 * 1 * 1)
        
        output = F.relu(self.fc1(output))
        output = F.relu(self.fc2(output))
        output = self.dp1(output)
        output = self.fc3(output)

        return output

class mush_rec_nn(nn.Module):
    def __init__(self):
        super(mush_rec_nn, self).__init__()
        
        k_s = 3
        pad = 0

        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=24, kernel_size=k_s, stride=1, padding=pad)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(24)
        
        self.cnn2 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=k_s, stride=1, padding=pad)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(48)
        
        self.cnn3 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=k_s, stride=1, padding=pad)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(96)
        
        self.cnn4 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=k_s, stride=1, padding=pad)
        self.pool4 = nn.MaxPool2d(2)
        self.bn4 = nn.BatchNorm2d(192)
        
        self.cnn5 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=k_s, stride=1, padding=pad)
        self.pool5 = nn.MaxPool2d(2)
        self.bn5 = nn.BatchNorm2d(384)
        
        linr = 384 * 1 * 1
        linr2 = linr
        self.fc1 = nn.Linear(linr, linr)
        self.fc2 = nn.Linear(linr, 384)
        self.dp1 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(384, N_CLASSES)
    
    def forward(self, input):
        output = self.pool1(F.relu(self.bn1(self.cnn1(input))))
        output = self.pool2(F.relu(self.bn2(self.cnn2(output))))  
        output = self.pool3(F.relu(self.bn3(self.cnn3(output))))
        output = self.pool4(F.relu(self.bn4(self.cnn4(output))))  
        output = self.pool5(F.relu(self.bn5(self.cnn5(output))))
        
        output = output.view(-1, 384 * 1 * 1)
        
        output = F.relu(self.fc1(output))
        output = self.fc2(output)
        output = self.dp1(output)
        output = self.fc3(output)

        return output
model = mush_rec_nn()

In [ ]:
path = './mush_rec_model_same_75%.pth'
path1 = './mush_rec_model_50%.pth'
model.load_state_dict(torch.load(path))
model.eval()

In [ ]:
img = Image.open('./training_set_t/Suillus/029_6_sz5LSbyBQ.jpg')

img = transform(img)
imageshow(img)
img = img[None,:]
img.shape

In [ ]:

dataset_ = datasets.ImageFolder('training_set_t', transform=transform)
N_CLASSES = len(dataset_.classes)
classes = dataset_.classes
print(dataset_.classes)

In [ ]:
train_set, val_set = random_split(dataset_, [0.8, 0.2])
trainloader, valloader = DataLoader(train_set, batch_size=64, shuffle=True), DataLoader(val_set, batch_size=64)

In [ ]:
def test_batch():
    for data in valloader:
        images, labels = data

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        for i, r_l, p_l in zip(range(len(images)), labels, predicted):
            if True:#r_l == p_l:
                imageshow(images[i])
                print(f"REAL: {classes[r_l]}, {r_l}")
                print(f"PREDICTED: {classes[p_l]}, {p_l}")
        break

def test_img():
    outputs = model(img)
    _, predicted = torch.max(outputs.data, 1)
    
    print(f"PREDICTED: {classes[predicted[0]]}")

In [ ]:
test_batch()